In [11]:
import os
import pandas as pd
import numpy as np
import difflib as diff

In [2]:
data_path = "/mnt/data/romy/hypnomed/MRI_raw"
dicom_path = os.path.join(data_path,'DICOM')
bids_path = os.path.join(data_path,'BIDS')

heudiconv_path = os.path.join(bids_path,'.heudiconv')
bids_info_paths = list()

for _,sub in enumerate(sorted(os.listdir(heudiconv_path))):
    bids_info_paths.append(os.path.join(heudiconv_path,'{}/ses-001/info/dicominfo_ses-001.tsv'.format(sub))) #permet de récupérer les paths des infos pour chaque sujet

### Retrive DICOM/BIDS infos for subjects

In [3]:
def get_dicom_infos(bids_info_paths,print_paths=False):

    dcm_dict_indxs = dict()
    infos_dict_indxs = dict()

    for sub_indx,bid_tsv in enumerate(bids_info_paths,1):
        if print_paths: #print the paths for sanitary check
            print(70*'-')
            print('Getting infos for subject n°{}, stored in {}'.format(sub_indx,bid_tsv))
            print(70*'-')
        infos = pd.read_csv(bid_tsv,sep='\t')
        infos_dict_indxs["Sub-{}".format(sub_indx)]=infos #store the tsv in df format for each subject
        dcm_dict_indxs["Sub-{}".format(sub_indx)]={}

        t2star_dcm = [type for type in infos.dcm_dir_name if 't2star' in type]
        rs_dcm = [type for type in infos.dcm_dir_name if 'rs' in type]
        t1_dcm = [type for type in infos.dcm_dir_name if '3DT1' in type]
        t2_dcm = [type for type in infos.dcm_dir_name if '3DT2' in type]

        dcm_dict = {'RS':rs_dcm,
                    'T1':t1_dcm,
                    'T2':t2_dcm,
                    'T2*':t2star_dcm}

        
        rs_indxs = list()
        t1_indxs = list()
        t2_indxs = list()
        t2star_indxs = list()
        
        for _,(key,val) in enumerate(zip(dcm_dict.keys(),dcm_dict.values())):
            if print_paths: #print the names of the files for sanitary check
                print("{} files : {}\n".format(key,val))
            if key == 'RS': #get the indx number that corresponds to each RS file :
                rs_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['RS'] = rs_indxs
            elif key == 'T1':
                t1_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['T1'] = t1_indxs
            elif key == 'T2':
                t2_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['T2'] = t2_indxs
            elif key == 'T2*':
                t2star_indxs.extend([val[i].split('_')[0] for i in range(len(val))])
                dcm_dict_indxs["Sub-{}".format(sub_indx)]['T2*'] = t2star_indxs
        
    return pd.DataFrame.from_dict(dcm_dict_indxs, orient='index'), infos_dict_indxs

In [4]:
select_subjects = [1,2,10] #None to run all, or ids of the subjetcs for whom we want to get the infos
select_subjects = None
if select_subjects:
    bids_info_paths_selected = list()
    bids_info_paths_selected.extend([path for (_,path) in enumerate(bids_info_paths,1) if _ in select_subjects]) 
    print('Selected files : ',bids_info_paths_selected)
    df_dicom_indx,infos_dict_indxs = get_dicom_infos(bids_info_paths=bids_info_paths_selected,print_paths=False)
else:
    df_dicom_indx,infos_dict_indxs = get_dicom_infos(bids_info_paths,print_paths=False)
    

df_dicom_indx
    

,RS,T1,T2,T2*
Sub-1,"[8, 16, 22]","[20, 21, 26, 27]","[14, 15]","[12, 13]"
Sub-2,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-3,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-4,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-5,"[14, 22, 28]","[26, 27]","[20, 21]","[18, 19]"
Sub-6,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-7,"[15, 21, 29]","[19, 20]","[27, 28]","[25, 26]"
Sub-8,"[8, 14, 22]","[12, 13]","[20, 21]","[18, 19]"
Sub-9,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-10,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"


In [9]:
print('T1',np.unique(df_dicom_indx['T1']))
print(70*'-')
print('T2',np.unique(df_dicom_indx['T2']))
print(70*'-')
print('T2*',np.unique(df_dicom_indx['T2*']))
print(70*'-')
print('RS',np.unique(df_dicom_indx['RS']))

T1 [list(['12', '13']) list(['13', '14']) list(['19', '20'])
 list(['20', '21']) list(['20', '21', '26', '27']) list(['22', '23'])
 list(['23', '24']) list(['26', '27']) list(['27', '28'])]
----------------------------------------------------------------------
T2 [list(['14', '15']) list(['16', '17']) list(['17', '18'])
 list(['20', '21']) list(['27', '28']) list(['32', '33'])]
----------------------------------------------------------------------
T2* [list(['12', '13']) list(['12', '13', '5001']) list(['14', '15'])
 list(['15', '16']) list(['18', '19']) list(['25', '26'])
 list(['30', '31'])]
----------------------------------------------------------------------
RS [list(['10', '18', '24']) list(['11', '19', '25'])
 list(['14', '22', '28']) list(['15', '21', '29']) list(['8', '14', '22'])
 list(['8', '16', '22']) list(['8', '16', '22', '101', '5003'])
 list(['8', '16', '29']) list(['8', '16', '29', '33'])
 list(['8', '18', '24']) list(['9', '15', '26', '34'])]


In [10]:
#d.DataFrame.from_dict(infos_dict_indxs, orient='index')

/home/romy.beaute/anaconda3/envs/hypnomed_env/lib/python3.9/site-packages/pandas/core/internals/construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0
Sub-1,total_files_till_now ...
Sub-2,total_files_till_now ...
Sub-3,total_files_till_now ...
Sub-4,total_files_till_now ...
Sub-5,total_files_till_now ...
Sub-6,total_files_till_now ...
Sub-7,total_files_till_now ...
Sub-8,total_files_till_now ...
Sub-9,total_files_till_now ...
Sub-10,total_files_till_now ...


### Get the differences between subjects datas

Using difflib : https://towardsdatascience.com/find-the-difference-in-python-68bbd000e513

In [12]:
df_dicom_indx

,RS,T1,T2,T2*
Sub-1,"[8, 16, 22]","[20, 21, 26, 27]","[14, 15]","[12, 13]"
Sub-2,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-3,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-4,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-5,"[14, 22, 28]","[26, 27]","[20, 21]","[18, 19]"
Sub-6,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-7,"[15, 21, 29]","[19, 20]","[27, 28]","[25, 26]"
Sub-8,"[8, 14, 22]","[12, 13]","[20, 21]","[18, 19]"
Sub-9,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
Sub-10,"[8, 16, 22]","[20, 21]","[14, 15]","[12, 13]"
